In [1]:
import sys 
sys.path.insert(0, '../')

In [2]:
pip install pytorch-pretrained-bert

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [10]:
from common.common import create_folder
from common.pytorch import load_model
import pytorch_pretrained_bert as Bert
from model.utils import age_vocab
from common.common import load_obj
from dataLoader.NumericalTrain import NTLoader
from torch.utils.data import DataLoader
import pandas as pd
from model.NT import BertForSequenceClassification
from sklearn.model_selection import train_test_split
from model.optimiser import adam
import sklearn.metrics as skm
import numpy as np
import torch
import time
import torch.nn as nn
import os

In [4]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        
class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

In [5]:
file_config = {
    'data': '',  # formated data 
    'model_path': 'model', # where to save model
    'model_name': 'SepsisBERT.csv', # model name
    'file_name': '',  # log path
}
create_folder(file_config['model_path'])

In [6]:
global_params = {
    'max_seq_len': 280,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
    'min_visit': 5,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 256,
    'use_cuda': False,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cpu'
}

In [7]:
data = pd.read_csv('../preprocess/final_dataset/final_dataset.csv')

In [8]:
data.head()

,HR,Temp,O2Sat,Resp,SepsisLabel,PatientID
0,"{6: 101.0, 10: 87.0, 14: 81.0, 18: 87.0, 22: 9...","{6: 36.6, 10: 36.0, 14: 36.0, 18: 36.7, 22: 36...","{6: 100.0, 10: 100.0, 14: 100.0, 18: 100.0, 22...","{6: 20.0, 10: 20.0, 14: 20.0, 18: 20.0, 22: 18...","{6: 0, 10: 0, 14: 0, 18: 0, 22: 0, 26: 0, 30: ...",p116812
1,"{0: 84.5, 1: 87.0, 2: 88.0, 3: 87.0, 4: 92.0, ...","{0: 34.3, 1: 34.6, 2: 35.5, 3: 36.2, 4: 36.65,...","{0: 96.0, 1: 96.0, 2: 95.0, 3: 98.0, 4: 98.5, ...","{0: 30.0, 1: 18.0, 2: 19.0, 3: 18.0, 4: 18.0, ...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",p109932
2,"{0: 80.0, 1: 76.0, 2: 80.0, 3: 78.0, 4: 74.0, ...","{0: 36.5, 1: 36.25, 2: 36.25, 3: 36.1, 4: 36.0...","{0: 100.0, 1: 100.0, 2: 100.0, 3: 100.0, 4: 10...","{0: 13.5, 1: 12.0, 2: 12.0, 3: 12.0, 4: 12.5, ...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",p014977
3,"{4: 83.0, 8: 81.0, 13: 85.0, 17: 92.0, 22: 87....","{4: 36.11, 8: 35.83, 13: 36.0, 17: 37.28, 22: ...","{4: 99.0, 8: 96.0, 13: 97.0, 17: 93.0, 22: 97....","{4: 16.0, 8: 17.0, 13: 19.0, 17: 17.0, 22: 15....","{4: 0, 8: 0, 13: 0, 17: 0, 22: 0, 28: 0, 39: 0}",p000902
4,"{0: 102.5, 5: 100.0, 8: 102.0, 17: 108.0, 18: ...","{0: 36.89, 5: 37.06, 8: 36.17, 17: 36.17, 18: ...","{0: 96.0, 5: 98.0, 8: 95.0, 17: 89.0, 18: 100....","{0: 16.5, 5: 14.5, 8: 16.0, 17: 18.0, 18: 15.0...","{0: 0, 5: 0, 8: 0, 17: 0, 18: 0, 20: 0, 25: 0,...",p009098


In [11]:
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [12]:
# Initialize the NTLoader for Training
Dset = NTLoader(train_df, max_len=global_params['max_seq_len'])
trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

# Initialize the NTLoader for Testing
Dset = NTLoader(test_df, max_len=global_params['max_seq_len'])
testload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

In [13]:
model_config = {
    'vocab_size': 10, # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'max_position_embedding': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 6, # number of multi-head attention layers required
    'num_attention_heads': 12, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [14]:
# Model Configuration
config = BertConfig(model_config)

# Initialize the model
num_features = 4  # Number of input features
model = BertForSequenceClassification(config, num_features=num_features)

In [15]:
#model = model.to(train_params['device'])
optim = adam(params=list(model.named_parameters()), config=optim_param)

t_total value of -1 results in schedule not being applied


In [16]:
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision

In [17]:
import time
import torch
from torch.optim import Adam

def train(epoch, loader, model, device, optimizer, print_step=256):
    model.train()
    tr_loss = 0
    temp_loss = 0
    start = time.time()

    for step, batch in enumerate(loader):
        # Move batch to the specified device
        features, labels = batch[0].to(device), batch[1].to(device)

        # Forward pass
        optimizer.zero_grad()
        loss, logits = model(features, labels=labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Logging
        temp_loss += loss.item()
        tr_loss += loss.item()

        elapsed = time.time() - start
        print(f"Epoch: {epoch} | Step: {step} | Avg Loss: {temp_loss / print_step:.4f} | Time Elapsed: {elapsed:.2f}s")
        temp_loss = 0
        start = time.time()

    # Calculate total loss for the epoch
    total_loss = tr_loss / len(loader)

    return total_loss

In [18]:
optimizer = Adam(model.parameters(), lr=0.001)
device = torch.device("cpu")

for e in range(2):
    loss = train(e, trainload, model, device, optimizer)
    print(f"Epoch {e} finished with loss: {loss:.4f}")

Epoch: 0 | Step: 0 | Avg Loss: 0.0022 | Time Elapsed: 21.34s
Epoch: 0 | Step: 1 | Avg Loss: 0.0000 | Time Elapsed: 8.02s
Epoch: 0 | Step: 2 | Avg Loss: 0.0000 | Time Elapsed: 8.12s
Epoch: 0 | Step: 3 | Avg Loss: 0.0000 | Time Elapsed: 7.63s
Epoch: 0 | Step: 4 | Avg Loss: 0.0000 | Time Elapsed: 7.77s
Epoch: 0 | Step: 5 | Avg Loss: 0.0000 | Time Elapsed: 8.94s
Epoch: 0 | Step: 6 | Avg Loss: 0.0000 | Time Elapsed: 9.30s
Epoch: 0 | Step: 7 | Avg Loss: 0.0000 | Time Elapsed: 8.75s
Epoch: 0 | Step: 8 | Avg Loss: 0.0000 | Time Elapsed: 7.58s
Epoch: 0 | Step: 9 | Avg Loss: 0.0000 | Time Elapsed: 7.78s
Epoch: 0 | Step: 10 | Avg Loss: 0.0000 | Time Elapsed: 8.07s
Epoch: 0 | Step: 11 | Avg Loss: 0.0000 | Time Elapsed: 7.67s
Epoch: 0 | Step: 12 | Avg Loss: 0.0000 | Time Elapsed: 7.70s
Epoch: 0 | Step: 13 | Avg Loss: 0.0000 | Time Elapsed: 8.02s
Epoch: 0 | Step: 14 | Avg Loss: 0.0000 | Time Elapsed: 7.62s
Epoch: 0 | Step: 15 | Avg Loss: 0.0000 | Time Elapsed: 7.61s
Epoch: 0 | Step: 16 | Avg Loss: 0

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch
import numpy as np

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_logits = []

    with torch.no_grad():  # No need to track gradients during evaluation
        for features, labels in test_loader:
            features = features.to(device)
            labels = labels.to(device)
            logits = model(features)  # Model outputs logits
            all_labels.extend(labels.cpu().numpy())
            all_logits.extend(logits.cpu().numpy())

    # Convert logits to probabilities for AUC calculation
    probabilities = torch.sigmoid(torch.tensor(all_logits)).numpy()

    # Flatten arrays (important if labels are sequences)
    all_labels_flat = np.concatenate(all_labels)
    probabilities_flat = np.concatenate(probabilities)

    # Calculate metrics
    accuracy = accuracy_score(all_labels_flat, np.round(probabilities_flat))
    precision = precision_score(all_labels_flat, np.round(probabilities_flat))
    recall = recall_score(all_labels_flat, np.round(probabilities_flat))
    f1 = f1_score(all_labels_flat, np.round(probabilities_flat))
    auc = roc_auc_score(all_labels_flat, probabilities_flat)

    return accuracy, precision, recall, f1, auc

In [26]:
accuracy, precision, recall, f1, auc = evaluate_model(model, testload, device)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}\nAUC-ROC: {auc}")

C:\Users\User\AppData\Local\Temp\ipykernel_5092\3392295728.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  probabilities = torch.sigmoid(torch.tensor(all_logits)).numpy()
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` para

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.